<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/startOverGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import google.generativeai as genai
import os
from google.colab import userdata
import pandas as pd
import time

In [6]:
api_key = userdata.get('JOE-GEMINI-KEY')
genai.configure(api_key=api_key)

model = genai.GenerativeModel('models/gemini-1.5-pro')

In [13]:
#most up to date
#handles everything

# Function to process a batch of messages
def detect(messages):
    prompts = [f"Is this text message real or fake? '{message}' Respond with one word: real, fake, or undetermined, without any explanations." for message in messages]
    response = model.generate_content(prompts)

    # Debugging: Print response object to understand its structure
    #print(f"Response: {response}")

    # Extract text from the response and split
    response_text = response.candidates[0].content.parts[0].text.strip()
    split_results = response_text.split('\n')

    # Further filter results to ensure only valid responses are considered
    valid_responses = ['real', 'fake', 'undetermined']
    results = [line.strip().lower() for line in split_results if line.strip().lower() in valid_responses]

    # Debugging: Print filtered results
    #print(f"Filtered results: {results}")

    return results

# Function to process the CSV file
def process_csv(input_file):
    # Load the data into a pandas DataFrame without headers
    df = pd.read_csv(input_file, header=None)

    # To store results for each value
    authenticity_results = []

    # Determine the number of tokens per message
    def count_tokens(text):
        return len(text.split())

    # Calculate tokens for each message
    df['tokens'] = df[0].apply(count_tokens)

    # Process rows in batches of 20 messages
    batch_size = 20
    max_requests_per_day = 50
    max_tokens_per_minute = 32000
    max_requests_per_minute = 2

    requests_made = 0
    total_tokens_processed = 0

    for i in range(0, len(df), batch_size):
        if requests_made >= max_requests_per_day:
            print("Daily limit reached. Stopping execution.")
            break

        batch = df[0][i:i+batch_size].tolist()
        batch_tokens = df['tokens'][i:i+batch_size].sum()

        if total_tokens_processed + batch_tokens > max_tokens_per_minute:
            print("Token limit per minute reached. Waiting for the next minute.")
            time.sleep(60)  # Wait for the next minute
            total_tokens_processed = 0  # Reset the token count for the new minute

        try:
            batch_results = detect(batch)
        except Exception as e:
            print(f"Error during detection: {e}")
            batch_results = ['undetermined'] * batch_size  # Fallback to undetermined for all in case of error

        if len(batch_results) != batch_size:
            print(f"Error: Expected {batch_size} results but got {len(batch_results)}. Continuing with next batch.")
            batch_results += ['undetermined'] * (batch_size - len(batch_results))  # Pad the results

        authenticity_results.extend(batch_results)
        total_tokens_processed += batch_tokens

        # Print status after processing each batch
        print(f"Processed inputs {i + 1} to {i + batch_size}, Results = {batch_results}")

        # Rate limiting: ensure only 2 requests per minute
        requests_made += 1
        if requests_made % max_requests_per_minute == 0:
            time.sleep(60)

    # Ensure the authenticity results list is the same length as the DataFrame
    if len(authenticity_results) < len(df):
        authenticity_results.extend([''] * (len(df) - len(authenticity_results)))

    # Assign results to new column
    df['authenticity'] = authenticity_results

    # Save the updated results back to the same file
    output_file = "processed_" + input_file
    df.to_csv(output_file, index=False, header=False)

    print("Processing complete. Results saved to", output_file)

# Example usage
input_file = '/content/sample_data/smishTankSet.csv'  # Update with your file path
process_csv(input_file)

Processed inputs 1 to 20, Results = ['fake', 'fake', 'fake', 'undetermined', 'undetermined', 'undetermined', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake']
Processed inputs 21 to 40, Results = ['fake', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake']
Error: Expected 20 results but got 21. Continuing with next batch.
Processed inputs 41 to 60, Results = ['fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'undetermined', 'undetermined', 'fake', 'undetermined', 'fake', 'fake']
Processed inputs 61 to 80, Results = ['fake', 'undetermined', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake']
Processed inputs 81 to 100, Results = ['fake',

KeyboardInterrupt: 

In [15]:
import pandas as pd
import time

# Function to process a batch of messages
def detect(messages):
    prompts = [f"Is this text message real or fake? '{message}' Respond with one word: real, fake, or undetermined, without any explanations." for message in messages]
    response = model.generate_content(prompts) #send to gemini and store in response

    # Extract text from the response and split
    response_text = response.candidates[0].content.parts[0].text.strip() #extracts text from response
    split_results = response_text.split('\n') #splits text into lines

    # Further filter results to ensure only valid responses are considered
    valid_responses = ['real', 'fake', 'undetermined']
    results = [line.strip().lower() for line in split_results if line.strip().lower() in valid_responses]

    # Ensure the number of results matches the number of messages
    if len(results) > len(messages):
        results = results[:len(messages)]
    elif len(results) < len(messages):
        results.extend(['undetermined'] * (len(messages) - len(results)))

    return results

# Function to process the CSV file
def process_csv(input_file):
    # Load the data into a pandas DataFrame without headers
    df = pd.read_csv(input_file, header=None)

    # To store results for each value
    authenticity_results = []

    # Determine the number of tokens per message
    def count_tokens(text):
        return len(text.split())

    # Calculate tokens for each message
    df['tokens'] = df[0].apply(count_tokens)

    # Process rows in batches of 20 messages
    batch_size = 20
    max_requests_per_day = 50
    max_tokens_per_minute = 32000
    max_requests_per_minute = 2

    requests_made = 0
    total_tokens_processed = 0

    for i in range(0, len(df), batch_size):
        if requests_made >= max_requests_per_day:
            print("Daily limit reached. Stopping execution.")
            break

        batch = df[0][i:i+batch_size].tolist()
        batch_tokens = df['tokens'][i:i+batch_size].sum()

        if total_tokens_processed + batch_tokens > max_tokens_per_minute:
            print("Token limit per minute reached. Waiting for the next minute.")
            time.sleep(60)  # Wait for the next minute
            total_tokens_processed = 0  # Reset the token count for the new minute

        try:
            batch_results = detect(batch)
        except Exception as e:
            print(f"Error during detection: {e}")
            batch_results = ['undetermined'] * batch_size  # Fallback to undetermined for all in case of error

        # Ensure batch_results has exactly batch_size elements
        if len(batch_results) != batch_size:
            print(f"Error: Expected {batch_size} results but got {len(batch_results)}. Adjusting results.")
            if len(batch_results) > batch_size:
                batch_results = batch_results[:batch_size]
            else:
                batch_results.extend(['undetermined'] * (batch_size - len(batch_results)))

        authenticity_results.extend(batch_results)
        total_tokens_processed += batch_tokens

        # Print status after processing each batch
        print(f"Processed inputs {i + 1} to {i + batch_size}, Results = {batch_results}")

        # Rate limiting: ensure only 2 requests per minute
        requests_made += 1
        if requests_made % max_requests_per_minute == 0:
            time.sleep(60)

    # Ensure the authenticity results list is the same length as the DataFrame
    if len(authenticity_results) < len(df):
        authenticity_results.extend([''] * (len(df) - len(authenticity_results)))

    # Assign results to new column
    df['authenticity'] = authenticity_results

    # Save the updated results back to the same file
    output_file = "processed_" + input_file
    df.to_csv(output_file, index=False, header=False)

    print("Processing complete. Results saved to", output_file)

# Example usage
input_file = '/content/sample_data/smishTankSet.csv'  # Update with your file path
process_csv(input_file)


Processed inputs 1 to 20, Results = ['fake', 'fake', 'fake', 'undetermined', 'undetermined', 'undetermined', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake']
Processed inputs 21 to 40, Results = ['fake', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake']
Processed inputs 41 to 60, Results = ['fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'undetermined', 'undetermined', 'fake', 'undetermined', 'fake']
Processed inputs 61 to 80, Results = ['fake', 'real', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake']
Processed inputs 81 to 100, Results = ['fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'f

KeyboardInterrupt: 

In [4]:
import pandas as pd
import time

# Function to process a batch of messages
def detect(messages):
    prompts = [f"Is this text message real or fake? '{message}' Respond with one word: real, fake, or undetermined, without any explanations." for message in messages]
    response = model.generate_content(prompts)

    # Extract text from the response and split
    response_text = response.candidates[0].content.parts[0].text.strip()
    split_results = response_text.split('\n')

    # Further filter results to ensure only valid responses are considered
    valid_responses = ['real', 'fake', 'undetermined']
    results = [line.strip().lower() for line in split_results if line.strip().lower() in valid_responses]

    # Log a warning if results length mismatch
    if len(results) != len(messages):
        print(f"Warning: Expected {len(messages)} results but got {len(results)}. Attempting to correct.")

    # Ensure the number of results matches the number of messages
    if len(results) > len(messages):
        results = results[:len(messages)]
    elif len(results) < len(messages):
        results.extend(['error'] * (len(messages) - len(results)))

    return results

# Function to process the CSV file
def process_csv(input_file):
    # Load the data into a pandas DataFrame without headers
    df = pd.read_csv(input_file, header=None)

    # To store results for each value
    authenticity_results = []

    # Determine the number of tokens per message
    def count_tokens(text):
        return len(text.split())

    # Calculate tokens for each message
    df['tokens'] = df[0].apply(count_tokens)

    # Process rows in batches of 20 messages
    batch_size = 20
    max_requests_per_day = 50
    max_tokens_per_minute = 32000
    max_requests_per_minute = 2

    requests_made = 0
    total_tokens_processed = 0

    for i in range(0, len(df), batch_size):
        if requests_made >= max_requests_per_day:
            print("Daily limit reached. Stopping execution.")
            break

        batch = df[0][i:i+batch_size].tolist()
        batch_tokens = df['tokens'][i:i+batch_size].sum()

        if total_tokens_processed + batch_tokens > max_tokens_per_minute:
            print("Token limit per minute reached. Waiting for the next minute.")
            time.sleep(60)  # Wait for the next minute
            total_tokens_processed = 0  # Reset the token count for the new minute

        try:
            batch_results = detect(batch)
        except Exception as e:
            print(f"Error during detection: {e}")
            batch_results = ['error'] * batch_size  # Fallback to error for all in case of error

        # Ensure batch_results has exactly batch_size elements
        if len(batch_results) != batch_size:
            print(f"Error: Expected {batch_size} results but got {len(batch_results)}. Adjusting results.")
            if len(batch_results) > batch_size:
                batch_results = batch_results[:batch_size]
            else:
                batch_results.extend(['error'] * (batch_size - len(batch_results)))

        authenticity_results.extend(batch_results)
        total_tokens_processed += batch_tokens

        # Print status after processing each batch
        print(f"Processed inputs {i + 1} to {i + batch_size}, Results = {batch_results}")

        # Rate limiting: ensure only 2 requests per minute
        requests_made += 1
        if requests_made % max_requests_per_minute == 0:
            time.sleep(60)

    # Ensure the authenticity results list is the same length as the DataFrame
    if len(authenticity_results) < len(df):
        authenticity_results.extend([''] * (len(df) - len(authenticity_results)))

    # Assign results to new column
    df['authenticity'] = authenticity_results

    # Save the updated results back to the same file
    output_file = "processed_" + input_file
    df.to_csv(output_file, index=False, header=False)

    print("Processing complete. Results saved to", output_file)

# Example usage
input_file = 'smishTankSet.csv'  # Update with your file path
process_csv(input_file)


Processed inputs 1 to 20, Results = ['fake', 'fake', 'fake', 'undetermined', 'fake', 'undetermined', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake']
Processed inputs 21 to 40, Results = ['fake', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake']
Processed inputs 41 to 60, Results = ['fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'undetermined', 'undetermined', 'fake', 'undetermined', 'fake']
Processed inputs 61 to 80, Results = ['fake', 'undetermined', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake']
Processed inputs 81 to 100, Results = ['fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'f

KeyboardInterrupt: 

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import pandas as pd
import time

# Function to process a batch of messages
def detect(messages):
    prompts = [f"Is this text message real or fake? '{message}' Respond with one word: real, fake, or undetermined, without any explanations." for message in messages]

    while True:
        response = model.generate_content(prompts)

        # Extract text from the response and split
        response_text = response.candidates[0].content.parts[0].text.strip()
        split_results = response_text.split('\n')

        # Further filter results to ensure only valid responses are considered
        valid_responses = ['real', 'fake', 'undetermined']
        results = [line.strip().lower() for line in split_results if line.strip().lower() in valid_responses]

        # Check if the results match the number of messages
        if len(results) == len(messages):
            return results
        else:
            print(f"Retrying: Expected {len(messages)} results but got {len(results)}. Retrying...")
            time.sleep(60)  # Sleep for 1 minute before retrying

# Function to process the CSV file
def process_csv(input_file):
    # Load the data into a pandas DataFrame without headers
    df = pd.read_csv(input_file, header=None)

    # To store results for each value
    authenticity_results = []

    # Determine the number of tokens per message
    def count_tokens(text):
        return len(text.split())

    # Calculate tokens for each message
    df['tokens'] = df[0].apply(count_tokens)

    # Process rows in batches of 20 messages
    batch_size = 20
    max_requests_per_day = 50
    max_tokens_per_minute = 32000
    max_requests_per_minute = 2

    requests_made = 0
    total_tokens_processed = 0

    for i in range(0, len(df), batch_size):
        if requests_made >= max_requests_per_day:
            print("Daily limit reached. Stopping execution.")
            break

        batch = df[0][i:i+batch_size].tolist()
        batch_tokens = df['tokens'][i:i+batch_size].sum()

        if total_tokens_processed + batch_tokens > max_tokens_per_minute:
            print("Token limit per minute reached. Waiting for the next minute.")
            time.sleep(60)  # Wait for the next minute
            total_tokens_processed = 0  # Reset the token count for the new minute

        try:
            batch_results = detect(batch)
        except Exception as e:
            print(f"Error during detection: {e}")
            batch_results = ['error'] * batch_size  # Fallback to error for all in case of error

        authenticity_results.extend(batch_results)
        total_tokens_processed += batch_tokens

        # Print status after processing each batch
        print(f"Processed inputs {i + 1} to {i + batch_size}, Results = {batch_results}")

        # Rate limiting: ensure only 2 requests per minute
        requests_made += 1
        if requests_made % max_requests_per_minute == 0:
            time.sleep(60)

    # Ensure the authenticity results list is the same length as the DataFrame
    if len(authenticity_results) < len(df):
        authenticity_results.extend([''] * (len(df) - len(authenticity_results)))

    # Assign results to new column
    df['authenticity'] = authenticity_results

    # Extract the directory from the input file path
    input_directory = os.path.dirname(input_file)
    output_file = os.path.join(input_directory, "processed_" + os.path.basename(input_file))

    # Save the updated results back to the same directory
    df.to_csv(output_file, index=False, header=False)

    print("Processing complete. Results saved to", output_file)

# Example usage
input_file = '/content/drive/My Drive/anti_smish_gpt/smishTankSet.csv'  # Update with your file path
process_csv(input_file)


Processed inputs 1 to 20, Results = ['fake', 'fake', 'fake', 'undetermined', 'undetermined', 'undetermined', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake']
Processed inputs 21 to 40, Results = ['fake', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake']
Retrying: Expected 20 results but got 21. Retrying...
Processed inputs 41 to 60, Results = ['fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'undetermined', 'fake', 'fake', 'undetermined', 'fake']
Processed inputs 61 to 80, Results = ['fake', 'real', 'fake', 'undetermined', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake']
Processed inputs 81 to 100, Results = ['fake', 'fake', 'fake', 'fake', 'fake', 'fak